In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
ticker_list = stock.get_market_ticker_list(market="KOSPI")

# precision_svm 값과 종목코드를 저장할 빈 리스트
precision_list = []

# 각 종목에 대해 반복
for ticker in tqdm(ticker_list, desc="Processing"):
    try:
        ticker_name = stock.get_market_ticker_name(ticker)
        start_date = "2015-03-20"
        end_date = "2021-03-20"
        # 주어진 기간 동안의 일별 거래량 정보 가져오기
        df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


        def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
            """MACD 및 MACD 신호 계산"""
            df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
            df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
            df['MACD'] = df['EMA_short'] - df['EMA_long']
            df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
            return df

        def calculate_rsi(df, window=14, signal_window=9):
            """RSI 및 RSI 신호 계산"""
            delta = df['종가'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # RSI 신호선 추가
            df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
            return df

        # MACD 및 MACD 신호 계산
        df = calculate_macd(df)

        df = calculate_rsi(df)

        # '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
        # df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

        # 지정된 기간에 대한 SMA 계산
        periods = [5, 20, 60, 120, 240]
        for period in periods:
            df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

        # 지정된 기간에 대한 EMA 계산
        for period in periods:
            df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

        # EMA 간의 차이 계산
        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

        for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
            df[f'{column}_등락률'] = df[column].pct_change() * 100

        # SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
        periods = [5, 20, 60, 120, 240]

        # SMA 등락률 계산 및 추가
        for period in periods:
            df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

        # EMA 등락률 계산 및 추가
        for period in periods:
            df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

        # 'MACD'가 0 이상일 때 1, 아니면 0을 할당
        df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

        # 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

        # 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

        # 시가 대비 종가 등락률 컬럼 추가
        df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
        df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
        df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
        df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
        # 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

        # 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


        #csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
        #df.to_csv(csv_file_path, encoding='utf-8-sig')

        #print(f"Data saved to {csv_file_path}")

        # 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
        df['next_day_return'] = df['등락률'].shift(-1)
        df['target'] = df['next_day_return'].apply(lambda x: 3 if x > 2 else (2 if x > -2 else 1))
        df.dropna(inplace=True)  # 마지막 행 삭제

        first_column_name = df.columns[0]

        df1= df.iloc[:-50, :]
        X = df1.drop(['next_day_return','target'], axis=1)
        y = df1['target']

        # Calculating correlations for Forward Selection
        correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
        correlations.columns = ['Feature', 'Correlation']

        top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

        # X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
        X_top8 = X[top_8_features]
        y = df1['target']

        # 데이터를 학습 세트와 테스트 세트로 분할
        X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)

        svm_model = SVC()
        svm_model.fit(X_train, y_train)
        y_pred_svm = svm_model.predict(X_test)

        # Precision score 계산
        precision_svm = precision_score(y_test, y_pred_svm, average='macro')
        precision_list.append({'Ticker': ticker, 'Name': ticker_name, 'Precision_SVM': precision_svm})
        pass
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# 결과를 데이터프레임으로 변환하고 precision_svm 값에 따라 내림차순 정렬
precision_df = pd.DataFrame(precision_list).sort_values(by='Precision_SVM', ascending=False)

# 데이터프레임 출력
precision_df


Processing:   3%|▎         | 28/953 [00:15<03:34,  4.32it/s] 

Error processing 375500: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Error processing 37550L: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   3%|▎         | 29/953 [00:15<03:11,  4.83it/s]

Error processing 37550K: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   3%|▎         | 33/953 [00:16<04:27,  3.45it/s]

...

Processing:  68%|██████▊   | 647/953 [03:28<02:12,  2.31it/s]

Error processing 457190: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:  69%|██████▉   | 656/953 [03:31<01:11,  4.14it/s]

Error processing 350520: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Error processing 334890: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:  71%|███████   | 675/953 [03:38<01:28,  3.15it/s]

In [6]:
csv_file_path = "C:/apps/h1/3targets_kospi_1503_2103_svmprecision_data.csv"  # 저장할 파일 경로 및 이름 설정
precision_df.to_csv(csv_file_path, encoding='utf-8-sig')